# Проект для интернет-магазина

## Подготовка

In [8]:
import torch
import transformers
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [9]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

In [10]:
display(comments.info())
display(comments.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Выделим признаки и целевой признак - toxic

In [11]:
target = comments['toxic']
features = comments['text']

Разделим датасет на обучающую и тестовую выборки.

In [17]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.2, shuffle=False)

Преобразуем данные к нужному типу.

In [34]:
corpus_train = features_train.values

In [35]:
corpus_test = features_test.values

Оставим в датасете только регулярные выражения. Воспользуемся методом re.sub(), применим его ко всему датасету

In [36]:
def clear_text(text):
    text_lem = re.sub(r'[^a-zA-Z]', ' ', text)
    return text_lem

In [37]:
for i in range(len(corpus_train)):
    corpus_train[i] = clear_text(corpus_train[i])

In [38]:
for i in range(len(corpus_test)):
    corpus_test[i] = clear_text(corpus_test[i])

С помощью стемминга приведем слова к форме основы. Воспользуемся инструментом EnglishStemmer из библиотеки nltk. Применим ко всему датасету.

In [39]:
stemmer = EnglishStemmer(ignore_stopwords=False)

In [40]:
%%time
for i in range(len(corpus_train)):
    word_list = nltk.word_tokenize(corpus_train[i])
    corpus_train[i] = ' '.join([stemmer.stem(w) for w in word_list])

CPU times: user 1min 46s, sys: 219 ms, total: 1min 47s
Wall time: 1min 47s


In [41]:
%%time
for i in range(len(corpus_test)):
    word_list = nltk.word_tokenize(corpus_test[i])
    corpus_test[i] = ' '.join([stemmer.stem(w) for w in word_list])

CPU times: user 26.7 s, sys: 55.2 ms, total: 26.8 s
Wall time: 26.8 s


Загрузим базу стоп-слов

In [42]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Векторизуем данные с учетом стоп-слов. Применим метод fit к обычающей выборке, а затем метод transform к обучающему и тестовому набору данных.

In [43]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(corpus_train)

In [44]:
train_x = tf_idf.transform(corpus_train)

In [45]:
test_x = tf_idf.transform(corpus_test)

Матрицы готовы, обучим на них модели

## Обучение

# LogisticRegression

In [46]:
model = LogisticRegression(random_state=12345, solver = 'liblinear', class_weight='balanced')

In [47]:
model.fit(train_x, target_train)

LogisticRegression(class_weight='balanced', random_state=12345,
                   solver='liblinear')

In [48]:
predicted = pd.Series(model.predict(test_x))

In [49]:
f1_score(predicted, target_test).round(2)

0.75

# Random Forest Classifier

Используем метод кросс-валидации для подбора гиперпараметров модели

In [53]:
#исследуем работу модели с различными значениями гиперпараметра max_depth
for max_depth in range(5, 26, 5):
    rf_model = RandomForestClassifier(class_weight = 'balanced', max_depth = max_depth, n_estimators = 100)
    rf_cv = cross_val_score(rf_model, train_x, target_train, cv=3,scoring='f1')
    print("Score при max_depth =", max_depth, ":", rf_cv)
    print("Score mean =", sum(rf_cv)/len(rf_cv))
    print()

Score при max_depth = 5 : [0.32775423 0.32857649 0.31931464]
Score mean = 0.32521512110120193

Score при max_depth = 10 : [0.33856109 0.36025567 0.35878234]
Score mean = 0.3525330320879141

Score при max_depth = 15 : [0.38828338 0.37879027 0.36143862]
Score mean = 0.3761707556864591

Score при max_depth = 20 : [0.39804546 0.38129228 0.38553694]
Score mean = 0.3882915615603882

Score при max_depth = 25 : [0.43001565 0.42040352 0.41945946]
Score mean = 0.42329287515685027



In [52]:
#исследуем работу модели с различными значениями гиперпараметра n_estimators
for estim in range(50, 201, 50):
    rf_model = RandomForestClassifier(class_weight = 'balanced', max_depth = 5, n_estimators = estim)
    rf_cv = cross_val_score(rf_model, train_x, target_train, cv=3,scoring='f1')
    print("Score при n_estimators =", estim, ":", rf_cv)
    print("Score mean =", sum(rf_cv)/len(rf_cv))
    print()

Score при n_estimators = 50 : [0.31530298 0.29994639 0.31231385]
Score mean = 0.3091877393417795

Score при n_estimators = 100 : [0.32679114 0.33567766 0.33988493]
Score mean = 0.334117909838656

Score при n_estimators = 150 : [0.33963614 0.32672029 0.32525217]
Score mean = 0.33053619659885086

Score при n_estimators = 200 : [0.34545866 0.3148997  0.34318276]
Score mean = 0.33451370773639394



Для обучения модели возьмем значение гиперпараметра max_depth = 25 и n_estimators = 200

In [56]:
model_1 = RandomForestClassifier(class_weight = 'balanced',max_depth = 25,n_estimators = 200)

In [57]:
model_1.fit(train_x, target_train)

RandomForestClassifier(class_weight='balanced', max_depth=25, n_estimators=200)

In [58]:
predicted_1 = pd.Series(model_1.predict(test_x))

In [59]:
f1_score(predicted_1, target_test).round(2)

0.42

## Выводы

Значение метрики F1 достигло необходимого значения только у модели LogisticRegression. Модель Random Forest Classifier не справилась с поставленной задачей.

Из-за большого количества наблюдений и их сложной структуры, мощности обычного компьютера периодически не хватает для полноценного анализа, из-за этого время обучения и работы возрастает, и не все модели можно детально рассмотреть.